In [64]:



class Pair:
    def __init__(self, children) -> None:
        self.children = []
        self.set_children(children)
        self.parent = None

    def set_children(self, children):
        if len(children) != 2:
            raise ValueError("Must have 2 children")
        self.children = children
        for child in children:
            if isinstance(child, self.__class__):
                child.parent = self

    def __eq__(self, other):
        return (
            self.__class__ == other.__class__
            and self.children == other.children
            and self.parent == other.parent
        )

    @classmethod
    def from_pair(cls, pair):
        children = []
        for item in pair:
            if isinstance(item, list):
                children.append(Pair.from_pair(item))
            else:
                children.append(item)
        return cls(children=children)

    def get_adjacent_number(self, idx):
        other_idx = abs(idx - 1)
        if self.parent.children[other_idx] == self:
            other = self.parent.children[idx]
            if isinstance(other, self.__class__):
                return other.get_first_number(other_idx)
            return other
        return self.parent.get_adjacent_number(idx)

    def get_first_number(self, idx):
        if isinstance(self.children[idx], self.__class__):
            return self.children[idx].get_first_number(idx)
        else:
            return self.children[idx]


In [149]:
import math

In [150]:
class Node:
    def __init__(self, value=None, pair=None) -> None:
        self.value = value
        self.parent = None
        self.left = None
        self.right = None
        if pair is not None:
            self.set_pair(*pair)

    # def __eq__(self, other):
    #     return (
    #         self.__class__ == other.__class__
    #         and self.value == other.value
    #         and self.parent == other.parent
    #         and self.left == other.left
    #         and self.right == other.right
    #     )

    def set_pair(self, left, right):
        if self.left is not None:
            self.left.parent = None
        if self.right is not None:
            self.right.parent = None
        self.left = left
        self.right = right
        self.left.parent = self
        self.right.parent = self

    def remove_pair(self):
        if self.left is not None:
            self.left.parent = None
            self.left = None
        if self.right is not None:
            self.right.parent = None
            self.right = None

    def __repr__(self) -> str:
        return f'Node<v={self.value} l={self.left is not None} r={self.right is not None}>'

    @classmethod
    def from_pair(cls, pair):
        children = []
        for item in pair:
            if isinstance(item, list):
                children.append(cls.from_pair(item))
            else:
                children.append(cls(value=item))
        return cls(pair=children)

    def get_number_to_left(self):
        if not self.parent:
            return None
        if self.parent.right is self:
            if self.parent.left.value is not None:
                return self.parent.left
            node = self.parent.left
            while node.right is not None:
                node = node.right
            return node
        return self.parent.get_number_to_left()

    def get_number_to_right(self):
        if not self.parent:
            return None
        if self.parent.left is self:
            if self.parent.right.value is not None:
                return self.parent.right
            node = self.parent.right
            while node.left is not None:
                node = node.left
            return node
        return self.parent.get_number_to_right()

    def explode(self):
        left2 = self.left.get_number_to_left()
        right2 = self.right.get_number_to_right()
        if left2:
            left2.value += self.left.value
        if right2:
            right2.value += self.right.value
        self.remove_pair()
        self.value = 0

    def split(self):
        if self.value is not None:
            raise ValueError("value must be set to split")
        left = Node(value=self.value // 2)
        right = Node(value=int(math.ceil(self.value / 2)))
        self.set_pair(left, right)
        self.value = None

In [140]:
p = Node.from_pair([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]])

In [141]:
c = p.left.right.right.right

In [143]:
c.explode()

In [147]:
c.get_number_to_right()

Node<v=9 l=False r=False>